In [27]:


import tiktoken
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import TokenTextSplitter
from llama_index import VectorStoreIndex, SimpleWebPageReader, ServiceContext, StorageContext
from llama_index.llms import OpenAI
from llama_index.node_parser import SimpleNodeParser
from llama_index.vector_stores import FaissVectorStore


In [2]:
import os

os.environ["OPENAI_API_KEY"] = "<Put the OpenAI API Key here>"

In [3]:
def get_documents(url: str):
    documents = SimpleWebPageReader(html_to_text=True).load_data(
        [url]
    )
    return documents


def num_tokens_from_string(string: str, model_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [5]:
def get_embedding():
    model_name = "sentence-transformers/all-MiniLM-l6-v2"
    model_kwargs = {'device': 'cpu'}

    encode_kwargs = {'normalize_embeddings': False}
    hf = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return hf

In [6]:
embedding_model = get_embedding()
documents = get_documents("https://www.pg.unicamp.br/norma/31594/0")

In [7]:
num_tokens_from_string(documents[0].text, "gpt-3.5-turbo")

65682

In [8]:
node_parser = SimpleNodeParser.from_defaults(
    text_splitter=TokenTextSplitter(chunk_size=4000, chunk_overlap=200)
)

llm = OpenAI(temperature=0, model="gpt-4", max_tokens=512)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_model,
    node_parser=node_parser
)

In [10]:
import faiss

d = 384
faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, service_context=service_context,
                                        show_progress=True)

Parsing documents into nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/24 [00:00<?, ?it/s]

In [11]:
nodes = node_parser.get_nodes_from_documents(documents)

In [12]:


from llama_index.prompts import (
    ChatMessage,
    ChatPromptTemplate,
    MessageRole,
    PromptTemplate,
)

import re

In [13]:
QA_PROMPT = PromptTemplate(
    "As informações dos documentos estão apresentadas abaixo.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Dadas as informações dos documentos e nenhum conhecimento prévio, "
    "responda a pergunta.\n"
    "Pergunta: {query_str}\n"
    "Resposta: "
)


def generate_answers_for_questions(questions, context, llm):
    """Generate answers for questions given context."""
    answers = []
    for question in questions:
        fmt_qa_prompt = QA_PROMPT.format(
            context_str=context, query_str=question
        )
        response_obj = llm.complete(fmt_qa_prompt)
        answers.append(str(response_obj))
    return answers

In [14]:
QUESTION_GEN_USER_TMPL = (
    "As informações dos documents estão apresentadas abaixo.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Dadas as informações dos documentos e nenhum conhecimento prévio, "
    "gere perguntas relevantes."
)

QUESTION_GEN_SYS_TMPL = (
    "Você é um professor de pesquisa. Usando os documentos fornecidos, "
    "sua tarefa é criar uma lista com APENAS 1 pergunta."
    "Limite as perguntas às informações fornecidas dos documentos.\n"
)

question_gen_template = ChatPromptTemplate(
    message_templates=[
        ChatMessage(role=MessageRole.SYSTEM, content=QUESTION_GEN_SYS_TMPL),
        ChatMessage(role=MessageRole.USER, content=QUESTION_GEN_USER_TMPL),
    ]
)


def generate_qa_pairs(nodes, llm, num_questions_per_chunk):
    """Generate questions."""
    qa_pairs = []
    for idx, node in enumerate(nodes):
        print(f"Node {idx}/{len(nodes)}")
        context_str = node.get_content(metadata_mode="all")
        fmt_messages = question_gen_template.format_messages(
            num_questions_per_chunk=num_questions_per_chunk,
            context_str=context_str,
        )
        chat_response = llm.chat(fmt_messages)
        raw_output = chat_response.message.content
        print(raw_output)
        result_list = str(raw_output).strip().split("\n")
        cleaned_questions = [
            re.sub(r"^\d+[\).\s]", "", question).strip()
            for question in result_list
        ]
        print(cleaned_questions)
        answers = generate_answers_for_questions(
            cleaned_questions, context_str, llm
        )
        print(answers)
        cur_qa_pairs = list(zip(cleaned_questions, answers))
        qa_pairs.extend(cur_qa_pairs)
    return qa_pairs

In [15]:
qa_pairs = generate_qa_pairs(
    nodes,
    llm,
    num_questions_per_chunk=1,
)

Node 0/24
Qual é o total de vagas oferecidas para o Vestibular Unicamp 2024 e como elas estão distribuídas entre os diferentes sistemas de ingresso?
['Qual é o total de vagas oferecidas para o Vestibular Unicamp 2024 e como elas estão distribuídas entre os diferentes sistemas de ingresso?']
['Para o ano de 2024, a Universidade Estadual de Campinas (Unicamp) está oferecendo um total de 3340 vagas regulares para ingresso nos Cursos de Graduação. Essas vagas estão distribuídas da seguinte forma entre os diferentes sistemas de ingresso:\n\n- 2537 vagas são oferecidas pelo Vestibular Unicamp (VU) 2024.\n- 314 vagas são oferecidas pelo Edital ENEM-Unicamp 2024.\n- 325 vagas são oferecidas pelo Provão Paulista 2024.\n- 49 vagas são oferecidas pelo Vestibular Indígena (VI) 2024, que terá ainda 81 vagas adicionais.\n- 115 vagas são oferecidas pelo Edital de olimpíadas científicas e competições de conhecimento de áreas específicas, que terá ainda 14 vagas adicionais.']
Node 1/24
Quais são as con

In [18]:
qa_pairs[0]

('Qual é o total de vagas oferecidas para o Vestibular Unicamp 2024 e como elas estão distribuídas entre os diferentes sistemas de ingresso?',
 'Para o ano de 2024, a Universidade Estadual de Campinas (Unicamp) está oferecendo um total de 3340 vagas regulares para ingresso nos Cursos de Graduação. Essas vagas estão distribuídas da seguinte forma entre os diferentes sistemas de ingresso:\n\n- 2537 vagas são oferecidas pelo Vestibular Unicamp (VU) 2024.\n- 314 vagas são oferecidas pelo Edital ENEM-Unicamp 2024.\n- 325 vagas são oferecidas pelo Provão Paulista 2024.\n- 49 vagas são oferecidas pelo Vestibular Indígena (VI) 2024, que terá ainda 81 vagas adicionais.\n- 115 vagas são oferecidas pelo Edital de olimpíadas científicas e competições de conhecimento de áreas específicas, que terá ainda 14 vagas adicionais.')

In [23]:
# save
import pickle

pickle.dump(qa_pairs, open("eval_dataset.pkl", "wb"))